Analyze the Contents of the Foundation Framework
=============================================
- Test file `@import Foundation;`
- How many headers?
- How many modules?
- How many AST Nodes
- Timing

In [1]:
using Clang, Clang_jll, DataFrames, DataFrameMacros, Revise

In [2]:
@show Clang_jll.libclang_path
@show sdk_dir = `xcrun --show-sdk-path` |> open |> readchomp |> String
@show clang()
nothing 

Clang_jll.libclang_path = "/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/libclang.dylib"
sdk_dir = ((`xcrun --show-sdk-path` |> open) |> readchomp) |> String = "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"
clang() = setenv(`/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/tools/clang`,["EDITOR=code", "PATH=/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/tools:/Users/nehal/.juliaup/bin:/Users/nehal/micromamba/condabin:/Users/nehal/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/usr/local/MacGPG2/bin:/usr/local/share/dotnet:~/.dotnet/tools:/Library/Apple/usr/bin:/Library/Frameworks/Mono.framework/Versions/Current/Commands:/Users/nehal/Library/Application Support/JetBrains/Toolbox/scripts:/opt/homebrew/opt/fzf/bin", "VSCODE_NLS_CONFIG={\"locale\":\"en-us\",\"availableLanguages\":{

In [30]:
sdk_dir

"/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"

In [3]:
run(`$(clang()) -x objective-c 050_import_module.h -emit-ast -o Foundation.pch -fmodules -isysroot $sdk_dir "-DNS_FORMAT_ARGUMENT(A)="`)

Process(setenv(`/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/tools/clang -x objective-c 050_import_module.h -emit-ast -o Foundation.pch -fmodules -isysroot /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk '-DNS_FORMAT_ARGUMENT(A)='`,["EDITOR=code", "PATH=/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/tools:/Users/nehal/.juliaup/bin:/Users/nehal/micromamba/condabin:/Users/nehal/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/usr/local/MacGPG2/bin:/usr/local/share/dotnet:~/.dotnet/tools:/Library/Apple/usr/bin:/Library/Frameworks/Mono.framework/Versions/Current/Commands:/Users/nehal/Library/Application Support/JetBrains/Toolbox/scripts:/opt/homebrew/opt/fzf/bin", "VSCODE_NLS_CONFIG={\"locale\":\"en-us\",\"availableLanguages\":{},\"_languagePackSupport\":true}", "MANPATH=/opt/homebrew/share/man::", "STARSHIP_S

In [4]:
index = Index()
tu = Clang.load_ast(index,"Foundation.pch")

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x0000000165714c10, Index(Ptr{Nothing} @0x00000001387540f0, 0, 1))

In [5]:
toplevel =  tu |> Clang.getTranslationUnitCursor |> children |> collect
@show size(toplevel)

size(toplevel) = (22297,)


(22297,)

In [6]:
@show cu = toplevel[3]
@show f = file(cu)
@show m = get_module(tu, f)
@show ast_file(m)
@show parent_module(m)
@show toplevel_headers(tu, m)
@show toplevel_headers(tu, parent_module(m))
@show is_system(m)
nothing

cu = toplevel[3] = CLCursor (CLFunctionDecl) cacosf(_Complex float)
f = file(cu) = CLFile ("/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h")
m = get_module(tu, f) = CLModule (Darwin.C.complex)
ast_file(m) = CLFile ("/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/C/clang/ModuleCache/2SUK4F9IFS0OK/Darwin-1IVCWVLR6MT9T.pcm")
parent_module(m) = CLModule (Darwin.C)
toplevel_headers(tu, m) = CLFile[CLFile ("/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h")]
toplevel_headers(tu, parent_module(m)) = CLFile[]
is_system(m) = true


In [7]:
toplevel_module(m)

CLModule (Darwin)

In [10]:
df = DataFrame()
df.cursor = toplevel
df

Row,cursor
,CLCursor
1,CLCursor (CLObjCClassRef) Protocol
2,CLCursor (CLTypedefDecl) max_align_t
3,CLCursor (CLFunctionDecl) cacosf(_Complex float)
4,CLCursor (CLFunctionDecl) cacos(_Complex double)
5,CLCursor (CLFunctionDecl) cacosl(_Complex long double)
6,CLCursor (CLFunctionDecl) casinf(_Complex float)
7,CLCursor (CLFunctionDecl) casin(_Complex double)
8,CLCursor (CLFunctionDecl) casinl(_Complex long double)
9,CLCursor (CLFunctionDecl) catanf(_Complex float)


In [11]:
df.file = df.cursor .|> file .|> name
df


Row,cursor,file
,CLCursor,String
1,CLCursor (CLObjCClassRef) Protocol,
2,CLCursor (CLTypedefDecl) max_align_t,/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1/include/__stddef_max_align_t.h
3,CLCursor (CLFunctionDecl) cacosf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
4,CLCursor (CLFunctionDecl) cacos(_Complex double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
5,CLCursor (CLFunctionDecl) cacosl(_Complex long double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
6,CLCursor (CLFunctionDecl) casinf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
7,CLCursor (CLFunctionDecl) casin(_Complex double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
8,CLCursor (CLFunctionDecl) casinl(_Complex long double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
9,CLCursor (CLFunctionDecl) catanf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h


In [20]:
df.mod = df.cursor .|> file .|> (f -> get_module(tu, f)) .|> full_name 
df

Row,cursor,file,mod
,CLCursor,String,String
1,CLCursor (CLObjCClassRef) Protocol,,
2,CLCursor (CLTypedefDecl) max_align_t,/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1/include/__stddef_max_align_t.h,_Builtin_stddef_max_align_t
3,CLCursor (CLFunctionDecl) cacosf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
4,CLCursor (CLFunctionDecl) cacos(_Complex double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
5,CLCursor (CLFunctionDecl) cacosl(_Complex long double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
6,CLCursor (CLFunctionDecl) casinf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
7,CLCursor (CLFunctionDecl) casin(_Complex double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
8,CLCursor (CLFunctionDecl) casinl(_Complex long double),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex
9,CLCursor (CLFunctionDecl) catanf(_Complex float),/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h,Darwin.C.complex


In [21]:
df.topmod = df.cursor .|> file .|> (f -> get_module(tu, f)) .|> toplevel_module .|> full_name 

22297-element Vector{String}:
 ""
 "_Builtin_stddef_max_align_t"
 "Darwin"
 "Darwin"
 "Darwin"
 "Darwin"
 "Darwin"
 "Darwin"
 "Darwin"
 "Darwin"
 ⋮
 "Foundation"
 "Foundation"
 "Foundation"
 "Foundation"
 "Foundation"
 "Foundation"
 "Foundation"
 "Foundation"
 ""

In [24]:
sort!(combine(groupby(df, :topmod), nrow),:nrow, rev=true)

Row,topmod,nrow
,String,Int64
1,Darwin,6316
2,CoreServices,3732
3,Security,3003
4,Foundation,2624
5,IOKit,2096
6,CoreFoundation,1661
7,CoreGraphics,1127
8,,355
9,CFNetwork,286


In [26]:
toplevel[3] |> kind |> spelling

"FunctionDecl"

In [27]:
df.kind = df.cursor .|> kind .|> spelling

22297-element Vector{String}:
 "ObjCClassRef"
 "TypedefDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 ⋮
 "VarDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "FunctionDecl"
 "ObjCCategoryDecl"
 "VarDecl"
 "FunctionDecl"
 "ModuleImport"

In [29]:
toplevel[end]

CLCursor (CLModuleImportDecl) Foundation

In [28]:
sort!(combine(groupby(df, :kind), nrow),:nrow, rev=true)

Row,kind,nrow
,String,Int64
1,FunctionDecl,8206
2,TypedefDecl,4716
3,VarDecl,3261
4,StructDecl,2596
5,EnumDecl,2327
6,ObjCClassRef,575
7,ObjCInterfaceDecl,274
8,ObjCCategoryDecl,181
9,ObjCProtocolDecl,66


Next Steps
======================
- All cursors
- Look for annotations
- language
- outliers 
  - Unexposed
- How to extract data (Clang Api)
  - documents
  - annotations
  - etc
- unions
- void * typedefs  